In [40]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import googlemaps
import gmaps
import requests
API_KEY = 'your_apikey'

import gmaps_func as gf
import datetime as dt

In [2]:
path = 'crime_data_clean/crime_15E40_10_17.csv'
df = pd.read_csv(path,index_col='Date_Hour', parse_dates=True)

In [3]:
df.head()

,Beat,BlockRange,StreetName,OffenseType,Premise,NumOffenses,Hour,day,weekday,month,year
Date_Hour,,,,,,,,,,,
2010-01-01 23:00:00,1.500000e+41,2500-2599,WESTRIDGE,Aggravated Assault,apartment,1,23,1,Friday,Jan,2010
2010-01-01 23:00:00,1.500000e+41,8300-8399,KNIGHT,Theft,convenience store,1,23,1,Friday,Jan,2010
2010-01-02 00:00:00,1.500000e+41,3000-3099,MURWORTH,Theft,other/unknown,1,0,2,Saturday,Jan,2010
2010-01-02 12:00:00,1.500000e+41,1600-1699,SOUTH LP W,Theft,hotel/motel/etc.,1,12,2,Saturday,Jan,2010
2010-01-03 00:00:00,1.500000e+41,2200-2299,HOLLY HALL,Auto Theft,apartment parking lot,1,0,3,Sunday,Jan,2010


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 16386 entries, 2010-01-01 23:00:00 to 2017-12-31 16:00:00
Data columns (total 11 columns):
Beat           16386 non-null float64
BlockRange     16386 non-null object
StreetName     16386 non-null object
OffenseType    16386 non-null object
Premise        16386 non-null object
NumOffenses    16386 non-null int64
Hour           16386 non-null int64
day            16386 non-null int64
weekday        16386 non-null object
month          16386 non-null object
year           16386 non-null int64
dtypes: float64(1), int64(4), object(6)
memory usage: 1.5+ MB


## Return a good blockrange value

In [5]:
gf.block_split(df)

ValueError: invalid literal for int() with base 10: 'UNK'

## Drop 'UNK' values

In [6]:
unk = df.BlockRange == 'UNK'

In [7]:
df = df[~unk]

In [8]:
df.head()

,Beat,BlockRange,StreetName,OffenseType,Premise,NumOffenses,Hour,day,weekday,month,year
Date_Hour,,,,,,,,,,,
2010-01-01 23:00:00,1.500000e+41,2500-2599,WESTRIDGE,Aggravated Assault,apartment,1,23,1,Friday,Jan,2010
2010-01-01 23:00:00,1.500000e+41,8300-8399,KNIGHT,Theft,convenience store,1,23,1,Friday,Jan,2010
2010-01-02 00:00:00,1.500000e+41,3000-3099,MURWORTH,Theft,other/unknown,1,0,2,Saturday,Jan,2010
2010-01-02 12:00:00,1.500000e+41,1600-1699,SOUTH LP W,Theft,hotel/motel/etc.,1,12,2,Saturday,Jan,2010
2010-01-03 00:00:00,1.500000e+41,2200-2299,HOLLY HALL,Auto Theft,apartment parking lot,1,0,3,Sunday,Jan,2010


In [9]:
df['block'] = gf.block_split(df)

/home/salas/miniconda3/envs/springboard/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 16281 entries, 2010-01-01 23:00:00 to 2017-12-31 16:00:00
Data columns (total 12 columns):
Beat           16281 non-null float64
BlockRange     16281 non-null object
StreetName     16281 non-null object
OffenseType    16281 non-null object
Premise        16281 non-null object
NumOffenses    16281 non-null int64
Hour           16281 non-null int64
day            16281 non-null int64
weekday        16281 non-null object
month          16281 non-null object
year           16281 non-null int64
block          16281 non-null object
dtypes: float64(1), int64(4), object(7)
memory usage: 2.2+ MB


## Create full addres col

In [11]:
df['address']  = df[['block', 'StreetName']].apply(lambda x: ' '.join(x), axis=1)

/home/salas/miniconda3/envs/springboard/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [12]:
df.head()

,Beat,BlockRange,StreetName,OffenseType,Premise,NumOffenses,Hour,day,weekday,month,year,block,address
Date_Hour,,,,,,,,,,,,,
2010-01-01 23:00:00,1.500000e+41,2500-2599,WESTRIDGE,Aggravated Assault,apartment,1,23,1,Friday,Jan,2010,2550,2550 WESTRIDGE
2010-01-01 23:00:00,1.500000e+41,8300-8399,KNIGHT,Theft,convenience store,1,23,1,Friday,Jan,2010,8350,8350 KNIGHT
2010-01-02 00:00:00,1.500000e+41,3000-3099,MURWORTH,Theft,other/unknown,1,0,2,Saturday,Jan,2010,3050,3050 MURWORTH
2010-01-02 12:00:00,1.500000e+41,1600-1699,SOUTH LP W,Theft,hotel/motel/etc.,1,12,2,Saturday,Jan,2010,1650,1650 SOUTH LP W
2010-01-03 00:00:00,1.500000e+41,2200-2299,HOLLY HALL,Auto Theft,apartment parking lot,1,0,3,Sunday,Jan,2010,2250,2250 HOLLY HALL


In [ ]:
df.info()

In [ ]:
df.head()

# get full lat long and address from google

In [48]:
df_4000 = df.iloc[:4000].copy()
df_8000 = df.iloc[4000:8000].copy()
df_12000 = df.iloc[8000:12000].copy()
df_16281  = df.iloc[12000:].copy()

In [34]:
len(df_4000)

4000

In [35]:
len(df_8000)

4000

In [49]:
len(df_12000)

4000

In [37]:
len(df_16281)

4281

In [32]:
df_4000.tail(2)

,Beat,BlockRange,StreetName,OffenseType,Premise,NumOffenses,Hour,day,weekday,month,year,block,address
Date_Hour,,,,,,,,,,,,,
2012-01-08 23:00:00,1.500000e+41,2100-2199,HOLLY HALL,Theft,apartment parking lot,1,23,8,Sunday,Jan,2012,2150,2150 HOLLY HALL
2012-01-09 22:00:00,1.500000e+41,8200-8299,CAMBRIDGE,Theft,apartment,1,22,9,Monday,Jan,2012,8250,8250 CAMBRIDGE


In [33]:
df_8000.head(2)

,Beat,BlockRange,StreetName,OffenseType,Premise,NumOffenses,Hour,day,weekday,month,year,block,address
Date_Hour,,,,,,,,,,,,,
2012-01-10 00:00:00,1.500000e+41,8000-8099,STADIUM,Theft,other/unknown,1,0,10,Tuesday,Jan,2012,8050,8050 STADIUM
2012-01-10 03:00:00,1.500000e+41,1400-1499,OLD SPANISH,Theft,hotel/motel/etc.,1,3,10,Tuesday,Jan,2012,1450,1450 OLD SPANISH


In [22]:
df_1200.tail(2)

,Beat,BlockRange,StreetName,OffenseType,Premise,NumOffenses,Hour,day,weekday,month,year,block,address
Date_Hour,,,,,,,,,,,,,
2016-02-07 13:00:00,1.500000e+41,9500-9599,MAIN,Robbery,bar or night club parking lot,1,13,7,Sunday,Feb,2016,9550,9550 MAIN
2016-02-07 15:00:00,1.500000e+41,3000-3099,MURWORTH,Burglary,apartment,1,15,7,Sunday,Feb,2016,3050,3050 MURWORTH


In [24]:
df_16281.tail(2)

,Beat,BlockRange,StreetName,OffenseType,Premise,NumOffenses,Hour,day,weekday,month,year,block,address
Date_Hour,,,,,,,,,,,,,
2017-12-31 12:00:00,1.500000e+41,8000-8099,BOSPHORUS,Theft,other parking lot,1,12,31,Sunday,Dec,2017,8050,8050 BOSPHORUS
2017-12-31 16:00:00,1.500000e+41,4400-4499,JUNE,Robbery,"toys, arts craft,musical, bike,pet stores",1,16,31,Sunday,Dec,2017,4450,4450 JUNE


In [26]:
df.tail(2)

,Beat,BlockRange,StreetName,OffenseType,Premise,NumOffenses,Hour,day,weekday,month,year,block,address
Date_Hour,,,,,,,,,,,,,
2017-12-31 12:00:00,1.500000e+41,8000-8099,BOSPHORUS,Theft,other parking lot,1,12,31,Sunday,Dec,2017,8050,8050 BOSPHORUS
2017-12-31 16:00:00,1.500000e+41,4400-4499,JUNE,Robbery,"toys, arts craft,musical, bike,pet stores",1,16,31,Sunday,Dec,2017,4450,4450 JUNE


## Test

In [38]:
df_20 = df.iloc[:20].copy()
df_20['tup_add'] = df_20['address'].apply(gf.gm_geocode,args=(API_KEY,))
df_20.head()

,Beat,BlockRange,StreetName,OffenseType,Premise,NumOffenses,Hour,day,weekday,month,year,block,address,tup_add
Date_Hour,,,,,,,,,,,,,,
2010-01-01 23:00:00,1.500000e+41,2500-2599,WESTRIDGE,Aggravated Assault,apartment,1,23,1,Friday,Jan,2010,2550,2550 WESTRIDGE,"((29.6823316, -95.4169951), 2550 Westridge St,..."
2010-01-01 23:00:00,1.500000e+41,8300-8399,KNIGHT,Theft,convenience store,1,23,1,Friday,Jan,2010,8350,8350 KNIGHT,"((29.6845315, -95.40031289999999), 8350 Knight..."
2010-01-02 00:00:00,1.500000e+41,3000-3099,MURWORTH,Theft,other/unknown,1,0,2,Saturday,Jan,2010,3050,3050 MURWORTH,"((29.6856544, -95.4256395), 3050 Murworth Dr, ..."
2010-01-02 12:00:00,1.500000e+41,1600-1699,SOUTH LP W,Theft,hotel/motel/etc.,1,12,2,Saturday,Jan,2010,1650,1650 SOUTH LP W,"((29.6792911, -95.403154), 1650 S Loop W, Hous..."
2010-01-03 00:00:00,1.500000e+41,2200-2299,HOLLY HALL,Auto Theft,apartment parking lot,1,0,3,Sunday,Jan,2010,2250,2250 HOLLY HALL,"((29.68490329999999, -95.3970744), 2250 Holly ..."


## Attempt

In [73]:
bf = dt.datetime.now().time().strftime("%X")
print('start:{}'.format(bf))
df_4000['tup_add'] = df_4000['address'].apply(gf.gm_geocode,args=(API_KEY,))

af = dt.datetime.now().time().strftime("%X")
print('end:{}'.format(af))

start:21:11:19
end:21:33:09


In [74]:
df_4000.head()

,Beat,BlockRange,StreetName,OffenseType,Premise,NumOffenses,Hour,day,weekday,month,year,block,address,tup_add
Date_Hour,,,,,,,,,,,,,,
2010-01-01 23:00:00,1.500000e+41,2500-2599,WESTRIDGE,Aggravated Assault,apartment,1,23,1,Friday,Jan,2010,2550,2550 WESTRIDGE,"((29.6823316, -95.4169951), 2550 Westridge St,..."
2010-01-01 23:00:00,1.500000e+41,8300-8399,KNIGHT,Theft,convenience store,1,23,1,Friday,Jan,2010,8350,8350 KNIGHT,"((29.6845315, -95.40031289999999), 8350 Knight..."
2010-01-02 00:00:00,1.500000e+41,3000-3099,MURWORTH,Theft,other/unknown,1,0,2,Saturday,Jan,2010,3050,3050 MURWORTH,"((29.6856544, -95.4256395), 3050 Murworth Dr, ..."
2010-01-02 12:00:00,1.500000e+41,1600-1699,SOUTH LP W,Theft,hotel/motel/etc.,1,12,2,Saturday,Jan,2010,1650,1650 SOUTH LP W,"((29.6792911, -95.403154), 1650 S Loop W, Hous..."
2010-01-03 00:00:00,1.500000e+41,2200-2299,HOLLY HALL,Auto Theft,apartment parking lot,1,0,3,Sunday,Jan,2010,2250,2250 HOLLY HALL,"((29.68490329999999, -95.3970744), 2250 Holly ..."


In [75]:
len(df_4000[df_4000.tup_add.isnull()])

0

In [76]:
df_4000.to_csv('crime_data_clean/crime_15E40_10_17df_4000.csv')

In [79]:
bf = dt.datetime.now().time().strftime("%X")
print('start:{}'.format(bf))
df_8000['tup_add'] = df_8000['address'].apply(gf.gm_geocode,args=(API_KEY,))

af = dt.datetime.now().time().strftime("%X")
print('end:{}'.format(af))

start:21:36:16
end:21:57:34


In [80]:
len(df_8000[df_8000.tup_add.isnull()])

0

In [81]:
df_8000.to_csv('crime_data_clean/crime_15E40_10_17df_8000.csv')

In [50]:
bf = dt.datetime.now().time().strftime("%X")
print('start:{}'.format(bf))
df_12000['tup_add'] = df_12000['address'].apply(gf.gm_geocode,args=(API_KEY,))

af = dt.datetime.now().time().strftime("%X")
print('end:{}'.format(af))

start:20:10:57
end:20:33:58


In [51]:
df_12000.head(2)

,Beat,BlockRange,StreetName,OffenseType,Premise,NumOffenses,Hour,day,weekday,month,year,block,address,tup_add
Date_Hour,,,,,,,,,,,,,,
2013-12-15 06:00:00,1.500000e+41,2100-2199,HOLLY HALL,Theft,apartment parking lot,1,6,15,Sunday,Dec,2013,2150,2150 HOLLY HALL,"((29.6851906, -95.3985319), 2150 Holly Hall St..."
2013-12-15 09:00:00,1.500000e+41,2500-2599,HOLLY HALL,Theft,residence/house,1,9,15,Sunday,Dec,2013,2550,2550 HOLLY HALL,"((29.6849626, -95.38957889999999), 2550 Holly ..."


In [52]:
bf = dt.datetime.now().time().strftime("%X")
print('start:{}'.format(bf))
df_16281['tup_add'] = df_16281['address'].apply(gf.gm_geocode,args=(API_KEY,))

af = dt.datetime.now().time().strftime("%X")
print('end:{}'.format(af))

start:20:35:14
end:21:01:01


In [82]:
len(df_8000[df_8000.tup_add.isnull()])

0

In [68]:
len(df_12000[df_12000.tup_add.isnull()])

0

In [69]:
df_12000.to_csv('crime_data_clean/crime_15E40_10_17df_12000.csv')

In [66]:
len(df_16281[df_16281.tup_add.isnull()])

0

In [67]:
df_16281.to_csv('crime_data_clean/crime_15E40_10_17df_16281.csv')

In [83]:
df_final = pd.concat([df_4000,df_8000,df_12000,df_16281])

In [84]:
len(df_final[df_final.tup_add.isnull()])

0

In [85]:
df_final.to_csv('crime_data_clean/crime_15E40_10_17ADD.csv')

In [86]:
len(df_final[df_final.tup_add.isnull()])

0

In [87]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 16281 entries, 2010-01-01 23:00:00 to 2017-12-31 16:00:00
Data columns (total 14 columns):
Beat           16281 non-null float64
BlockRange     16281 non-null object
StreetName     16281 non-null object
OffenseType    16281 non-null object
Premise        16281 non-null object
NumOffenses    16281 non-null int64
Hour           16281 non-null int64
day            16281 non-null int64
weekday        16281 non-null object
month          16281 non-null object
year           16281 non-null int64
block          16281 non-null object
address        16281 non-null object
tup_add        16281 non-null object
dtypes: float64(1), int64(4), object(9)
memory usage: 1.9+ MB


In [88]:
new_add = df_final.tup_add.apply(pd.Series,index=['lat_lon','full_address'])

In [89]:
new_add.head()

,lat_lon,full_address
Date_Hour,,
2010-01-01 23:00:00,"(29.6823316, -95.4169951)","2550 Westridge St, Houston, TX 77054, USA"
2010-01-01 23:00:00,"(29.6845315, -95.40031289999999)","8350 Knight Rd, Houston, TX 77054, USA"
2010-01-02 00:00:00,"(29.6856544, -95.4256395)","3050 Murworth Dr, Houston, TX 77025, USA"
2010-01-02 12:00:00,"(29.6792911, -95.403154)","1650 S Loop W, Houston, TX 77054, USA"
2010-01-03 00:00:00,"(29.68490329999999, -95.3970744)","2250 Holly Hall St, Houston, TX 77054, USA"


In [92]:
df_final = df_final.join(new_add)

In [93]:
df_final.head(2)

,Beat,BlockRange,StreetName,OffenseType,Premise,NumOffenses,Hour,day,weekday,month,year,block,address,tup_add,lat_lon,full_address
Date_Hour,,,,,,,,,,,,,,,,
2010-01-01 23:00:00,1.500000e+41,2500-2599,WESTRIDGE,Aggravated Assault,apartment,1,23,1,Friday,Jan,2010,2550,2550 WESTRIDGE,"((29.6823316, -95.4169951), 2550 Westridge St,...","(29.6823316, -95.4169951)","2550 Westridge St, Houston, TX 77054, USA"
2010-01-01 23:00:00,1.500000e+41,2500-2599,WESTRIDGE,Aggravated Assault,apartment,1,23,1,Friday,Jan,2010,2550,2550 WESTRIDGE,"((29.6823316, -95.4169951), 2550 Westridge St,...","(29.6845315, -95.40031289999999)","8350 Knight Rd, Houston, TX 77054, USA"


In [94]:
df_final.drop(['block','BlockRange','StreetName','address','tup_add','address','tup_add',],axis=1,inplace=True)

In [95]:
df_final.head(2)

,Beat,OffenseType,Premise,NumOffenses,Hour,day,weekday,month,year,lat_lon,full_address
Date_Hour,,,,,,,,,,,
2010-01-01 23:00:00,1.500000e+41,Aggravated Assault,apartment,1,23,1,Friday,Jan,2010,"(29.6823316, -95.4169951)","2550 Westridge St, Houston, TX 77054, USA"
2010-01-01 23:00:00,1.500000e+41,Aggravated Assault,apartment,1,23,1,Friday,Jan,2010,"(29.6845315, -95.40031289999999)","8350 Knight Rd, Houston, TX 77054, USA"


In [96]:
df_final.to_csv('crime_data_clean/crime_15E40_10_17FINAL.csv')